<a href="https://colab.research.google.com/github/derek-byte/Melanoma_Deep_Learning/blob/main/Web_App_Melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=1fbfd8cd5325decbe26d35ca40c41b549535ed72b0a07d50f69d325ca570ec7e
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken 26myWhdKL3FJvSiQfVOLduesolf_3iGZpQGynAH14orGSyv9z

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%%writefile app.py

import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobile 
import tensorflow_hub as hub
import matplotlib.pylab as plt
import os
import random

labels = ['Melanoma', 'NotMelanoma']

from PIL import Image, ImageOps
@st.cache
def load_image(image_file):
  img = Image.open(image_file)
  return img

def upload(filename):
    img = cv2.imread(filename)
    img = cv2.resize(img, (224, 224) )
    img = img /255
    return img

st.title("Melanoma Detection")

path = "/content/drive/MyDrive/Melanoma_Models_Good_Copy"
reloaded = tf.keras.models.load_model(path, custom_objects={'KerasLayer':hub.KerasLayer})
def predict_reload(image):
    probabilities = reloaded.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    return {labels[class_idx]: probabilities[class_idx]}

uploaded_file = st.file_uploader("Choose an image file", type=["png", "jpeg", "jpg", "svg"])

if uploaded_file is not None:
  # Convert the file to an opencv image
  file_bytes = np.asarray(bytearray(uploaded_file.read()),dtype=np.uint8)
  opencv_image = cv2.imdecode(file_bytes, 1)
  opencv_image = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
  resized = cv2.resize(opencv_image, (224,224))
  st.image(load_image(uploaded_file), width=224)

  # Saving File
  complete_path = os.path.join("/content/Melanoma_Images", uploaded_file.name)

  # CREATE THIS FILE SOURCE EVERYTIME ON GOOGLE COLAB
  with open(complete_path, "wb") as f:
    f.write(uploaded_file.getbuffer())
  st.success("File Saved")

  # Download to google drive?

  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest',
      validation_split=0.3)
  
  validation_generator = train_datagen.flow_from_directory(
    "/content/Melanoma_Images", 
    color_mode="rgb", 
    shuffle=False, 
    seed=None,
    class_mode="categorical",
    subset="validation",
    target_size=(224,224),
    batch_size=1)

  img = upload(complete_path)
  prediction = predict_reload(img)
  st.success("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))

Writing app.py


In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='80')
print(public_url)
!streamlit run --server.port 80 app.py >/dev/null


NgrokTunnel: "http://2e6e-35-229-87-228.ngrok.io" -> "http://localhost:80"
2022-04-06 14:16:09.634 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
2022-04-06 14:16:19.017071: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-06 14:16:19.025 Using /tmp/tfhub_modules to cache modules.


In [6]:
!pgrep streamlit